# Reproduce Brian Long's results with Starfish

The `brian_long_smFISH.zip` file needed to follow along with this notebook can be downloaded [here]()

This notebook walks through a work flow that reproduces the smFISH result for one field of view using the starfish package. 
It assumes that you have unzipped `brian_long_smFISH.zip` in the same directory as this notebook. Thus, you should see:

raw/
brian_long_smFISH.ipynb

In [1]:
import os
from starfish.io import Stack
import numpy as np

In [4]:
cat $experiment_json

{"version": "0.0.0", "hybridization_images": "hybridization.json", "auxiliary_images": {"nuclei": {"file": "img_000000000_Bandpass405_000.tif", "tile_shape": [2048, 2048], "tile_format": "TIFF", "coordinates": {"x": [0.0, 0.0001], "y": [0.0, 0.0001]}}}}

In [80]:
# package this up same as before
experiment_json = os.path.expanduser('/Users/ajc/google_drive/starfish/data/brian_long_smFISH/experiment.json')

In [81]:
s = Stack()
s.read(experiment_json)

In [4]:
# brian's data is uint, but starfish loads it as floats. Cast it to int here, which does not cause any loss of precision. 
s.image._data = s.image._data.astype(np.uint16)
istack = s.image

In [7]:
# this doesn't work, do we want this abstraction? 
# s.image.dtype

OrderedDict([(<Indices.HYB: 'hyb'>, 1),
             (<Indices.CH: 'ch'>, 3),
             (<Indices.Z: 'z'>, 34),
             ('y', 2048),
             ('x', 2048)])

Image processing function list:

1. clip & floor (at 10th percentile)
  1. `starfish.transform.threshold` or `.clip`
  2. Also called _after_ bandpass. 
2. trackpy bandpass
  1. `filters.bandpass`
4. gaussian filter (over z) 
  1. `starfish.filters.gaussian`. The skimage function supports nd images, so we just need to adjust the object in starfish. 
5. call peak locations (trackpy locate)
  1. `starfish.starfish.spots.crocker-grier` OR `starfish.starfish.spots.gaussian` (second class in this file)